# WSB SCRAPER

## 1) Load packages

In [ ]:
import praw
import pandas as pd
from textblob import TextBlob
import nltk
# nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

## 2) Set up Reddit API

In [ ]:
reddit = praw.Reddit(client_id='dwvhQN_PoUCoAw',
                     client_secret='X8N_SZUsiI-CNVIYLToBFFQ-cYE',
                     user_agent='news on hooks')

## 3) Retrieve data

### 3.1) Retrieve submissions

In [ ]:
submissions = reddit.subreddit('wallstreetbets').new(limit = 100)

### 3.2) Parse submissions

In [ ]:
sub_data = pd.DataFrame(columns = ['id', 'title', 'author', 'score', 'flair', 'ncomms', 'created', 'body'])
for sub in submissions:
    sub_data = sub_data.append(
    pd.Series(
    [
        sub.id, 
        sub.title, 
        sub.author,
        sub.score,
        sub.link_flair_text, 
        sub.num_comments,
        sub.created_utc,
        sub.selftext
    ], 
    index = ['id', 'title', 'author', 'score', 'flair', 'ncomms', 'created', 'body']
    ),
        ignore_index = True
    )
sub_data

### 3.3) Retrieve comment forrest

In [ ]:
com_data =  pd.DataFrame(columns = ['id', 'subid', 'parentid', 'author', 'score', 'created', 'body'])
for s_id in sub_data.id:
    sub = reddit.submission(id=s_id)
    print(sub.title)
    sub.comments.replace_more(limit=None)
    comment_queue = sub.comments[:]  # Seed with top-level
    while comment_queue:
        comment = comment_queue.pop(0)
        com_data = com_data.append(
        pd.Series(
        [
            comment.id, 
            comment.link_id,
            comment.parent_id,
            comment.author,
            comment.score,
            comment.created_utc,
            comment.body
        ],
            index = ['id', 'subid', 'parentid', 'author', 'score', 'created', 'body']
        ),
            ignore_index = True
        )
        comment_queue.extend(comment.replies)
        
com_data